*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
TRAIN_DATA_URL = "https://raw.githubusercontent.com/beaucarnes/fcc_python_curriculum/master/sms/train-data.tsv"
TEST_DATA_URL = "https://raw.githubusercontent.com/beaucarnes/fcc_python_curriculum/master/sms/valid-data.tsv"

train_file_path = tf.keras.utils.get_file("train-data.tsv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("valid-data.tsv", TEST_DATA_URL)

In [ ]:
# convert data to pandas dataframe and change labels to numbers
df_train = pd.read_csv(train_file_path,sep='\t',header=None)
df_test = pd.read_csv(test_file_path,sep='\t',header=None)
df_train[0] = df_train[0].replace("ham", 0)
df_train[0] = df_train[0].replace("spam", 1)
df_test[0] = df_test[0].replace("ham", 0)
df_test[0] = df_test[0].replace("spam", 1)
df_test[0]=df_test[0].astype('int64')
df_train[0]=df_train[0].astype('int64')
df_train.head()

In [ ]:
# convert data to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((df_train[1], df_train[0]))
test_dataset = tf.data.Dataset.from_tensor_slices((df_test[1], df_test[0]))

In [ ]:
# build vocab list
tokenizer = tfds.deprecated.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in train_dataset.concatenate(test_dataset):
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

In [ ]:
# create encoder based on vocab list
encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set)

In [ ]:
# encode datasets
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

def encode_map_fn(text, label):
  encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.int64))
  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label


train_dataset_encoded = train_dataset.map(encode_map_fn)
test_dataset_encoded = test_dataset.map(encode_map_fn)

In [ ]:
# check what data looks like after encoding
for train_example, train_label in train_dataset_encoded.take(2):
  print('Encoded text:', train_example[:10].numpy())
  print('Label:', train_label.numpy())

In [ ]:
# prepare data for training, padded_batch is used to make all reviews the same length while batching
BUFFER_SIZE = 1000

train_batches = (
    train_dataset_encoded
    .shuffle(BUFFER_SIZE)
    .padded_batch(32))

test_batches = (
    test_dataset_encoded
    .padded_batch(32))

In [ ]:
# build neural network model
model = keras.Sequential([
  keras.layers.Embedding(encoder.vocab_size, 16),
  keras.layers.GlobalAveragePooling1D(),
  keras.layers.Dense(1, activation='sigmoid')])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
# train model
history = model.fit(train_batches,
                    epochs=10,
                    validation_data=test_batches,
                    validation_steps=30)

In [ ]:
# evaluate model
loss, accuracy = model.evaluate(test_batches)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  encoded_pred_text = encoder.encode(pred_text)
  encoded_pred_text = tf.cast(encoded_pred_text, tf.float32)
  prediction = model.predict(tf.expand_dims(encoded_pred_text, tf.constant(0))).tolist()
  prediction = prediction[0]
  if prediction[0] < .5:
    prediction.append("ham")
  else:
    prediction.append("spam")

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
